In [170]:
import numpy as np
import pandas as pd

# membaca file csv
df = pd.read_csv("datasetfix.csv")
# menampilkan seluruh kolom yang tersedia
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0.1,Unnamed: 0,title,kategori,penilaian_ulasan,operating_hours_per_week,distance_km
0,1,26,Laundry88,Laundry,4.7,84.0,2.586678
1,2,27,Laundry Club Gegerkalong (Laundry Antar Jemput),Laundry,4.9,91.0,6.632171
2,3,28,Waroenk Laundry Tubagus Ismail,Laundry,4.3,98.0,3.578865
3,4,34,Sorcha Laundry Kosambi (Laundry Antar Jemput B...,Laundry,5.0,91.0,0.956055


In [171]:
# Mendefinisikan kriteria
kriteria = ['penilaian_ulasan', 'operating_hours_per_week', 'distance_km']

# Mendefinisikan matriks perbandingan berpasangan untuk kriteria
kriteria_matrik = np.array([
    [1, 1/2, 3],
    [2, 1, 4],
    [1/3, 1/4, 1]
])
kriteria_matrik

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 4.        ],
       [0.33333333, 0.25      , 1.        ]])

In [172]:
# Normalisasi matriks perbandingan berpasangan
def normalisasi_matrik(matrik):
    jumlah_kolom = np.sum(matrik, axis=0)
    return matrik / jumlah_kolom

In [ ]:
# Normalisasi dan menghitung bobot kriteria
norm_kriteria_matrik = normalisasi_matrik(kriteria_matrik)
nilai_eigen, vektor_eigen = np.linalg.eig(norm_kriteria_matrik)

vektor_eigen_utama = vektor_eigen[:, np.argmax(nilai_eigen)]
vektor_eigen_utama /= np.sum(vektor_eigen_utama)  # Normalisasi

print("Vektor Eigen Utama (Bobot):", vektor_eigen_utama)

Vektor Eigen Utama (Bobot): [0.30141844+0.j 0.57092199+0.j 0.12765957+0.j]


In [ ]:
# Fungsi untuk menghitung Indeks Konsistensi (CI)
def menghitung_ci(matrix, vektor_eigen):
    n = matrix.shape[0]
    lambda_max = np.max(np.linalg.eigvals(matrix))
    ci = (lambda_max - n) / (n - 1)
    return ci

# Fungsi untuk menghitung Rasio Konsistensi (CR)
def menghitung_cr(ci, ri):
    cr = ci / ri
    return cr

In [ ]:
# Indeks Acak (RI) untuk n=3 adalah 0.58
ri_kriteria = 0.58

# Menghitung CI dan CR untuk kriteria
ci_kriteria = menghitung_ci(kriteria_matrik, vektor_eigen_utama)
cr_kriteria = menghitung_cr(ci_kriteria, ri_kriteria)

In [176]:
# Mendapatkan nilai dari dataset untuk tiap kriteria dan alternatif
alternatives = df['title'].tolist()  # Mengambil nama alternatif
review_ratings = df['penilaian_ulasan'].values
operating_hours = df['operating_hours_per_week'].values
distances = df['distance_km'].values

In [177]:
# Mengonversi data ke dalam bentuk matriks perbandingan untuk setiap kriteria
matrik_review = np.array([
    review_ratings / review_ratings[j] for j in range(len(review_ratings))
])
matrik_hours = np.array([
    operating_hours / operating_hours[j] for j in range(len(operating_hours))
])
matrik_distance = np.array([
    distances / distances[j] for j in range(len(distances))
])

In [178]:
# Matriks Penilaian Ulasan
matrik_review = np.array([
    [1, 1/2, 4, 1/3],
    [2, 1, 6, 1/2],
    [1/4, 1/6, 1, 1/7],
    [3, 2, 7, 1]
])

# Jam Operasional per Minggu
matrik_hours = np.array([
    [1, 1/3, 1/7, 1/5],
    [3, 1, 1/5, 1/4],
    [7, 5, 1, 3],
    [5, 4, 1/3, 1]
])

# Jarak (semakin rendah semakin baik)
matrik_distance = np.array([
    [1, 3, 5, 7],
    [1/3, 1, 3, 5],
    [1/5, 1/3, 1, 3],
    [1/7, 1/5, 1/3, 1]
])

# Normalisasi dan hitung eigenvector untuk masing-masing matriks
norm_review = normalisasi_matrik(matrik_review)
nilai_eigen_review, vektor_eigen_review = np.linalg.eig(norm_review)
vektor_eigen_utama_review = vektor_eigen_review[:, np.argmax(nilai_eigen_review)]
vektor_eigen_utama_review /= np.sum(vektor_eigen_utama_review)

norm_hours = normalisasi_matrik(matrik_hours)
nilai_eigen_hours, vektor_eigen_hours = np.linalg.eig(norm_hours)
vektor_eigen_utama_hours = vektor_eigen_hours[:, np.argmax(nilai_eigen_hours)]
vektor_eigen_utama_hours /= np.sum(vektor_eigen_utama_hours)

norm_distance = normalisasi_matrik(matrik_distance)
nilai_eigen_distance, vektor_eigen_distance = np.linalg.eig(norm_distance)
vektor_eigen_utama_distance = vektor_eigen_distance[:, np.argmax(nilai_eigen_distance)]
vektor_eigen_utama_distance /= np.sum(vektor_eigen_utama_distance)

In [179]:
# Menghitung skor akhir untuk setiap alternatif berdasarkan bobot kriteria
skor_akhir = (
    vektor_eigen_utama_review * vektor_eigen_utama[0] +
    vektor_eigen_utama_hours * vektor_eigen_utama[1] +
    vektor_eigen_utama_distance * vektor_eigen_utama[2]
)

In [180]:
# Mengurutkan alternatif berdasarkan skor akhir
rank_alternatif = [alternatives[i] for i in np.argsort(skor_akhir)[::-1]]

print("Bobot Akhir:", skor_akhir)
print("Urutan Alternatif:", rank_alternatif)
print()

# Menghitung CI dan CR untuk setiap kriteria
ci_review = menghitung_ci(matrik_review, vektor_eigen_utama_review)
cr_review = menghitung_cr(ci_review, 0.58)  # RI untuk n=4 adalah 0.9
print("Indeks Konsistensi Review (CI):", ci_review)
print("Rasio Konsistensi Review (CR):", cr_review)
print()

ci_hours = menghitung_ci(matrik_hours, vektor_eigen_utama_hours)
cr_hours = menghitung_cr(ci_hours, 0.9)  # RI untuk n=4 adalah 0.9
print("Indeks Konsistensi Hours (CI):", ci_hours)
print("Rasio Konsistensi Hours (CR):", cr_hours)
print()

ci_distance = menghitung_ci(matrik_distance, vektor_eigen_utama_distance)
cr_distance = menghitung_cr(ci_distance, 0.9)  # RI untuk n=4 adalah 0.9
print("Indeks Konsistensi Distance (CI):", ci_distance)
print("Rasio Konsistensi Distance (CR):", cr_distance)
print()

Bobot Akhir: [0.1636447 +0.j 0.17236797+0.j 0.36984836+0.j 0.29413896+0.j]
Urutan Alternatif: ['Waroenk Laundry Tubagus Ismail', 'Sorcha Laundry Kosambi (Laundry Antar Jemput Bandung)', 'Laundry Club Gegerkalong (Laundry Antar Jemput)', 'Laundry88']

Indeks Konsistensi Review (CI): (0.015632005844133563+0j)
Rasio Konsistensi Review (CR): (0.026951734214023384+0j)

Indeks Konsistensi Hours (CI): (0.05694333232534691+0j)
Rasio Konsistensi Hours (CR): (0.06327036925038546+0j)

Indeks Konsistensi Distance (CI): (0.03899414781575672+0j)
Rasio Konsistensi Distance (CR): (0.04332683090639636+0j)

